# Cleaning and preparing an image dataset using fastdup V1.0

In [ ]:
# download fastdup
!pip install pip -U
!pip install fastdup
!pip install pandas

In [6]:
import fastdup
import pandas as pd

In [13]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04.5 LTS"
NAME="Ubuntu"
VERSION="20.04.5 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.5 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


# Download food-101 Dataset

In [1]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xf food-101.tar.gz

--2023-03-01 06:49:43--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2023-03-01 06:49:43--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  18.6MB/s    in 4m 22s  

2023-03-01 06:54:06 (18.2 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



# Run fastdup

In [7]:
images_dir = 'food-101/images/'
work_dir = 'fastdup_food101'

In [12]:
# fd = fastdup.create(work_dir=work_dir, input_dir=images_dir)
fd.run()

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
fastdup C++ error received:  read_features_parallel allowed values 1..64
 



NoneType: None
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/fastdup/sentry.py", line 114, in inner_function
    ret = func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/fastdup/fastdup_controller.py", line 300, in run
    self._create_img_mapping()
  File "/usr/local/lib/python3.8/dist-packages/fastdup/fastdup_controller.py", line 617, in _create_img_mapping
    df_mapping = self._fetch_df(FD.MAPPING_CSV).reset_index()
AttributeError: 'NoneType' object has no attribute 'reset_index'


AttributeError: ignored

# Find duplicates

In [ ]:
# visualize clusters of duplicate images
fd.vis.component_gallery(max_width=800)

In [ ]:
# get a list of clusters with duplicate images using connected components
cc_df, _ = fd.connected_components()
cc_df[cc_df['count'] > 0.0].sort_values(by=['count'], ascending=False).head()

,fastdup_id,component_id,sum,count,mean_distance,min_distance,max_distance,img_filename,error_code,is_valid
26532,26532,26474,5.8554,6.0,0.9759,0.9742,0.9791,crab_cakes/2780623.jpg,VALID,True
83939,83939,83758,5.8665,6.0,0.9778,0.9675,0.9917,red_velvet_cake/754664.jpg,VALID,True
21854,21854,21811,6.0000,6.0,1.0000,1.0000,1.0000,chocolate_cake/49494.jpg,VALID,True
19862,19862,19819,5.8648,6.0,0.9775,0.9721,0.9856,chicken_quesadilla/535546.jpg,VALID,True
19861,19861,19819,5.8648,6.0,0.9775,0.9721,0.9856,chicken_quesadilla/535532.jpg,VALID,True


## Connected Components Grouping

In [ ]:
# a function to group connected components
def get_clusters(df, sort_by='count', min_count=2, ascending=False):
    # columns to aggregate
    agg_dict = {'img_filename': list, 'mean_distance': max, 'count': len}

    if 'label' in df.columns:
        agg_dict['label'] = list
    
    # filter by count
    df = df[df['count'] >= min_count]
    
    # group and aggregate columns
    grouped_df = df.groupby('component_id').agg(agg_dict)
    
    # sort
    grouped_df = grouped_df.sort_values(by=[sort_by], ascending=ascending)
    return grouped_df

In [ ]:
# get clusters using connected components
cluster_df = get_clusters(cc_df)
cluster_df.head(3)

,img_filename,mean_distance,count
component_id,,,
19343,"[chicken_quesadilla/2388429.jpg, chicken_quesadilla/3004094.jpg, chicken_quesadilla/3779974.jpg]",0.9795,3
23608,"[churros/3303267.jpg, churros/3303373.jpg, churros/3303522.jpg]",0.9681,3
33468,"[edamame/3024335.jpg, edamame/3028728.jpg, edamame/3112981.jpg]",0.9749,3


In [ ]:
# get clusters sorted differently
get_clusters(cc_df, sort_by='mean_distance').head(3)

,img_filename,mean_distance,count
component_id,,,
131,"[apple_pie/1461580.jpg, apple_pie/1469191.jpg]",1.0,2
9853,"[breakfast_burrito/662420.jpg, breakfast_burrito/662423.jpg, breakfast_burrito/662424.jpg]",1.0,3
20899,"[chicken_wings/820377.jpg, chicken_wings/834669.jpg]",1.0,2


In [ ]:
# It's a good start, but as we can see there are not that many duplicates here, and the data may contain more.
# Let's lower the threshold a bit and re-evaluate the duplicates case.

## Re-run with lower threshold
Now we have more clusters containing more of the images, and we are able to remove highly similar images with higher recall.

In [ ]:
# run with ccthreshold = 0.9 compared to default 0.96
fd.run(ccthreshold=0.9, overwrite=True)
fd.summary()


 ########################################################################################

Dataset Analysis Summary: 

    Dataset contains 101000 images
    Valid images are 100.00% (101,000) of the data, invalid are 0.00% (0) of the data
    Similarity:  1.69% (1,704) belong to 30 similarity clusters (components).
    98.31% (99,296) images do not belong to any similarity cluster.
    Largest cluster has 79 (0.08%) images.
    For a detailed analysis, use `.connected_components()`
(similarity threshold used is 0.9, connected component threshold used is 0.9).

    Outliers: 5.97% (6,031) of images are possible outliers, and fall in the bottom 5.00% of similarity values.
    For a detailed list of outliers, use `.outliers(data=True)`.


['Dataset contains 101000 images',
 'Valid images are 100.00% (101,000) of the data, invalid are 0.00% (0) of the data',
 'Similarity:  1.69% (1,704) belong to 30 similarity clusters (components).',
 '98.31% (99,296) images do not belong to any similarity cluster.',
 'Largest cluster has 79 (0.08%) images.',
 'For a detailed analysis, use `.connected_components()`\n(similarity threshold used is 0.9, connected component threshold used is 0.9).\n',
 'Outliers: 5.97% (6,031) of images are possible outliers, and fall in the bottom 5.00% of similarity values.',
 'For a detailed list of outliers, use `.outliers(data=True)`.']

In [ ]:
# Now we see a much larger amount of images clustered together
cc90_df, _ = fd.connected_components()
cc90_df[cc90_df['count'] > 0.0].sort_values(by=['count'], ascending=False).head()

,fastdup_id,component_id,sum,count,mean_distance,min_distance,max_distance,img_filename,error_code,is_valid
40453,40453,24819,73.0287,79.0,0.9244,0.9004,0.9467,french_fries/2700217.jpg,VALID,True
40088,40088,24819,73.0287,79.0,0.9244,0.9004,0.9467,french_fries/1384733.jpg,VALID,True
40979,40979,24819,73.0287,79.0,0.9244,0.9004,0.9467,french_fries/889641.jpg,VALID,True
40027,40027,24819,73.0287,79.0,0.9244,0.9004,0.9467,french_fries/1099260.jpg,VALID,True
40070,40070,24819,73.0287,79.0,0.9244,0.9004,0.9467,french_fries/1295274.jpg,VALID,True


In [ ]:
# let's see the new clusters
fd.vis.component_gallery(max_width=800)

## Get a list of duplicates to remove

In [ ]:
# List the largest clusters. These include non identical but larger clusters.
clusters_df = get_clusters(cc90_df)
clusters_df.head(3)

,img_filename,mean_distance,count
component_id,,,
24819,"[club_sandwich/1297247.jpg, club_sandwich/1318118.jpg, club_sandwich/1886101.jpg, club_sandwich/2778614.jpg, club_sandwich/3106065.jpg, club_sandwich/588478.jpg, french_fries/1099260.jpg, french_fries/1295274.jpg, french_fries/1361604.jpg, french_fries/1384733.jpg, french_fries/147628.jpg, french_fries/1610240.jpg, french_fries/1692353.jpg, french_fries/1700344.jpg, french_fries/1712331.jpg, french_fries/1740113.jpg, french_fries/1810352.jpg, french_fries/1969264.jpg, french_fries/2073415.jpg, french_fries/2246387.jpg, french_fries/2348229.jpg, french_fries/2369999.jpg, french_fries/2700217.jpg, french_fries/2761796.jpg, french_fries/2885926.jpg, french_fries/2936284.jpg, french_fries/3030853.jpg, french_fries/3069835.jpg, french_fries/3359887.jpg, french_fries/3405511.jpg, french_fries/3423618.jpg, french_fries/3499831.jpg, french_fries/3669402.jpg, french_fries/3673168.jpg, french_fries/3697215.jpg, french_fries/3746805.jpg, french_fries/3832957.jpg, french_fries/3907871.jpg, french_fries/467106.jpg, french_fries/889641.jpg]",0.9244,40
18236,"[chicken_curry/2394967.jpg, chicken_curry/2701143.jpg, chicken_curry/882723.jpg, hot_and_sour_soup/1151861.jpg, hot_and_sour_soup/1167380.jpg, hot_and_sour_soup/1400511.jpg, hot_and_sour_soup/1617113.jpg, hot_and_sour_soup/1670529.jpg, hot_and_sour_soup/2041812.jpg, hot_and_sour_soup/2367229.jpg, hot_and_sour_soup/2377494.jpg, hot_and_sour_soup/2520927.jpg, hot_and_sour_soup/3086202.jpg, hot_and_sour_soup/3113531.jpg, hot_and_sour_soup/3286625.jpg, hot_and_sour_soup/3428336.jpg, hot_and_sour_soup/3452669.jpg, hot_and_sour_soup/3552976.jpg, hot_and_sour_soup/3567487.jpg, hot_and_sour_soup/3568665.jpg, hot_and_sour_soup/3601021.jpg, hot_and_sour_soup/3706507.jpg, hot_and_sour_soup/387487.jpg, hot_and_sour_soup/478316.jpg, hot_and_sour_soup/564763.jpg, lobster_bisque/1346617.jpg, lobster_bisque/1826587.jpg, lobster_bisque/2917736.jpg, lobster_bisque/3282626.jpg, lobster_bisque/3319694.jpg, lobster_bisque/3358721.jpg, lobster_bisque/3414592.jpg, lobster_bisque/3466502.jpg]",0.9250,33
26403,"[crab_cakes/3467918.jpg, pad_thai/1709738.jpg, pad_thai/3059603.jpg, spaghetti_bolognese/3565695.jpg, spaghetti_carbonara/1117183.jpg, spaghetti_carbonara/1390373.jpg, spaghetti_carbonara/1559267.jpg, spaghetti_carbonara/1668631.jpg, spaghetti_carbonara/1739526.jpg, spaghetti_carbonara/1891700.jpg, spaghetti_carbonara/190173.jpg, spaghetti_carbonara/1936669.jpg, spaghetti_carbonara/1940255.jpg, spaghetti_carbonara/2228065.jpg, spaghetti_carbonara/2281641.jpg, spaghetti_carbonara/2567706.jpg, spaghetti_carbonara/2774715.jpg, spaghetti_carbonara/2796656.jpg, spaghetti_carbonara/2835081.jpg, spaghetti_carbonara/2967972.jpg, spaghetti_carbonara/2980079.jpg, spaghetti_carbonara/3045854.jpg, spaghetti_carbonara/3377897.jpg, spaghetti_carbonara/3581296.jpg, spaghetti_carbonara/3708340.jpg, spaghetti_carbonara/3908531.jpg, spaghetti_carbonara/560793.jpg, spaghetti_carbonara/733714.jpg, spaghetti_carbonara/755025.jpg]",0.9279,29


In [ ]:
# List the clusters with the highest similarity. These are identical but smaller clusters.
get_clusters(cc90_df, sort_by='mean_distance').head(3)

,img_filename,mean_distance,count
component_id,,,
131,"[apple_pie/1461580.jpg, apple_pie/1469191.jpg]",1.0,2
9786,"[breakfast_burrito/662420.jpg, breakfast_burrito/662423.jpg, breakfast_burrito/662424.jpg]",1.0,3
20767,"[chicken_wings/820377.jpg, chicken_wings/834669.jpg]",1.0,2


In [ ]:
# First sample from each cluster that is kept
cluster_images_to_keep = []
cluster_images_to_discard = []

for cluster_file_list in clusters_df.img_filename:
    # keep first file, discard rest
    keep = cluster_file_list[0]
    discard = cluster_file_list[1:]
    
    cluster_images_to_keep.append(keep)
    cluster_images_to_discard.extend(discard)

cluster_images_to_discard = set(cluster_images_to_discard)
print(f"Found {len(cluster_images_to_discard)} highly similar images to discard")

Found 2124 highly similar images to discard


# Find outliers

In [ ]:
# show furthest outliers
fd.outliers().head(5)

,index,outlier,nearest,distance,img_filename_outlier,error_code_outlier,is_valid_outlier,img_filename_nearest,error_code_nearest,is_valid_nearest
0,10099,9797,96131,0.379365,breakfast_burrito/462294.jpg,VALID,True,tacos/1505262.jpg,VALID,True
1,10098,63325,38290,0.429240,macarons/2117640.jpg,VALID,True,fish_and_chips/2079080.jpg,VALID,True
2,10095,96131,83667,0.515787,tacos/1505262.jpg,VALID,True,red_velvet_cake/3143813.jpg,VALID,True
3,10094,89017,25421,0.528563,shrimp_and_grits/1047420.jpg,VALID,True,club_sandwich/2465517.jpg,VALID,True
4,10093,95577,16743,0.530332,sushi/3100962.jpg,VALID,True,cheese_plate/3693838.jpg,VALID,True


In [ ]:
# visualize outliers
fd.vis.outliers_gallery()

# Remove broken images
Using fastdup we are able to recover a list of files that are corrupted or that could not be loaded from various reasons. The reason is listed for each image. We will fetch them, and add them to our list of images to remove. Food-101 is meticoulsly curated, so just for the sake of demonstration we've added one empty image.

In [ ]:
inv = fd.invalid_instances()
inv

,img_filename,fastdup_id,error_code,is_valid


In [ ]:
inv = pd.DataFrame([['apple_pie/broken_image.jpg', 101001, 'ERROR_ZERO_SIZE_FILE', False]], columns=[inv.columns])
inv

,img_filename,fastdup_id,error_code,is_valid
0,apple_pie/broken_image.jpg,101001,ERROR_ZERO_SIZE_FILE,False


# Find dark, bright and blurry images

In [ ]:
# show image statistics
fd.img_stats().head(5)

,fastdup_id,img_w,img_h,unique,blur,mean,min,max,stdv,file_size,contrast,img_filename,error_code,is_valid
0,0,308,512,0,2538.7280,113.0935,0.0,255.0,53.7116,42196,1.0000,apple_pie/1005649.jpg,VALID,True
1,1,512,512,0,334.7137,131.6409,4.0,255.0,55.6296,40760,0.9691,apple_pie/1011328.jpg,VALID,True
2,2,384,512,0,728.2955,117.8109,0.0,255.0,29.5684,33835,1.0000,apple_pie/101251.jpg,VALID,True
3,3,512,512,0,1013.1310,95.8764,0.0,255.0,67.6263,48051,1.0000,apple_pie/1014775.jpg,VALID,True
4,4,512,512,0,2478.7454,63.2681,0.0,255.0,68.2799,59055,1.0000,apple_pie/1026328.jpg,VALID,True


In [ ]:
# visualize blurry images
fd.vis.stats_gallery(metric='blur')

In [ ]:
# visualize dark images
fd.vis.stats_gallery(metric='dark')

In [ ]:
# visualize bright images
fd.vis.stats_gallery(metric='bright')

# Wrap up and collect file list

In [ ]:
# let's collect all files to remove
stats_df = fd.img_stats()
outlier_df = fd.outliers()

# for outliers, we'll take images which their closest similarity is lower than 0.68
outliers_filtered = outlier_df[outlier_df.distance < 0.68].img_filename_outlier.tolist()

images_to_remove = dict(
    duplicates = list(cluster_images_to_discard), # the duplicates threshold is 0.9, the same ccthreshold we ran fastdup with
    outliers_to_discard = outliers_filtered,
    invalid_images = fd.invalid_instances(),
    blurry_images = stats_df[stats_df['blur'] < 50], # we'll choose blur threshold as 50
    dark_images = stats_df[stats_df['mean'] < 13], # darkness threshold as 13
    bright_images = stats_df[stats_df['mean'] > 220.5], # and brightness threshold as 220.5
)

In [ ]:
# we'll create a set of unique images to remove
images_to_discard = []

for key, data_slice in images_to_remove.items():
    ext = data_slice if isinstance(data_slice, list) else data_slice['img_filename'].tolist()
    print(f"{key}: {len(data_slice)}")
    images_to_discard.extend(ext)
print(f"Total - {len(set(images_to_discard))} unique images")

duplicates: 2124
outliers_to_discard: 175
invalid_images: 0
blurry_images: 30
dark_images: 6
bright_images: 13
Total - 2340 unique images


# Summary 
Even under restrictive thresholds, we have found 2,340 images that should not be included in our labeling or training using this data.